# Start spark application (spark session)

In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1649069323437_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
type(spark)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.session.SparkSession'>

# Load ticket data from the S3 bucket

In [3]:
s3_path = "s3a://upgrad-data/Parking_Violation_Tickets.csv"
tickets = spark.read.csv(s3_path, header = True, inferSchema = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Cache the data

In [4]:
tickets.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Summons Number: bigint, Plate ID: string, Registration State: string, Plate Type: string, Issue Date: string, Violation Code: int, Vehicle Body Type: string, Vehicle Make: string, Issuing Agency: string, Street Code1: int, Street Code2: int, Street Code3: int, Vehicle Expiration Date: int, Violation Location: int, Violation Precinct: int, Issuer Precinct: int, Issuer Code: int, Issuer Command: string, Issuer Squad: string, Violation Time: string, Time First Observed: string, Violation County: string, Violation In Front Of Or Opposite: string, House Number: string, Street Name: string, Intersecting Street: string, Date First Observed: int, Law Section: int, Sub Division: string, Violation Legal Code: string, Days Parking In Effect    : string, From Hours In Effect: string, To Hours In Effect: string, Vehicle Color: string, Unregistered Vehicle?: int, Vehicle Year: int, Meter Number: string, Feet From Curb: int, Violation Post Code: string, Violation Description: string, No Sta

# Check the cache status

In [5]:
tickets.is_cached

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [6]:
tickets.storageLevel.useMemory

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [7]:
tickets.storageLevel.useDisk

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

# Check schema of the DataFrame

In [8]:
tickets.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation Coun

# Total number of tickets

In [9]:
tickets.select(tickets["Summons Number"]).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10803028

# Create SQL view

In [10]:
tickets.createOrReplaceTempView("tickets_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Check for duplicate entries

In [11]:
spark.sql("\
            SELECT `Summons Number`, COUNT(`Summons Number`) AS `Number of duplicate occurance of summons number`\
            FROM tickets_view\
            GROUP BY `Summons Number`\
            HAVING COUNT(`Summons Number`) > 1\
        ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-----------------------------------------------+
|Summons Number|Number of duplicate occurance of summons number|
+--------------+-----------------------------------------------+
+--------------+-----------------------------------------------+

# Number of unique states from where cars got parking ticket

In [12]:
spark.sql("\
            SELECT COUNT(`Registration State`) as `Number of unique states`\
            FROM\
            (\
                SELECT `Registration State`\
                FROM tickets_view\
                GROUP BY `Registration State`\
            ) AS SUBQUERY\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|Number of unique states|
+-----------------------+
|                     67|
+-----------------------+